In [10]:
import pandas as pd    
test = pd.read_csv("./test_labels.csv")
train = pd.read_csv("./train_labels.csv")

In [11]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense,Lambda
from keras.utils import plot_model
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [12]:
classes = ['civilian aircraft','civilian car','military aircraft', 'military helicopter','military tank','military truck']

In [13]:
image_rows, image_cols = 256, 256

In [5]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = [image_rows, image_cols,3]),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(64, (2, 2), activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(64, (2, 2), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(len(classes), activation ='softmax')
])
model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics=['accuracy'])

In [10]:
train_dir="./Images"
train_csv_path="./train_labels.csv"
train_df=train

test_csv_path="./test_labels.csv"
test_df=test
test_dir="./Images"

batch_size=32

img_size=(image_rows, image_cols)

gen=ImageDataGenerator(rescale=1/255, validation_split = 0.25)


In [11]:
train_gen=gen.flow_from_dataframe(train_df, directory=train_dir, x_col='filename', y_col='class', shuffle=True,
                                 target_size=img_size, class_mode='categorical', batch_size=batch_size, subset='training')

Found 8037 validated image filenames belonging to 6 classes.


E:\Anaconda\lib\site-packages\keras\preprocessing\image.py:989: UserWarning: Found 3 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn('Found {} invalid image filename(s) in x_col="{}". '


In [12]:
valid_gen=gen.flow_from_dataframe(train_df, directory=train_dir, x_col='filename', y_col='class', shuffle=True,
                                 target_size=img_size, class_mode='categorical', batch_size=batch_size, subset='validation')


Found 2678 validated image filenames belonging to 6 classes.


E:\Anaconda\lib\site-packages\keras\preprocessing\image.py:989: UserWarning: Found 3 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn('Found {} invalid image filename(s) in x_col="{}". '


In [13]:
test_gen=gen.flow_from_dataframe(test_df, directory=test_dir, x_col='filename', y_col='class', shuffle=False,
                                 target_size=img_size, class_mode='categorical', batch_size=batch_size)      

Found 718 validated image filenames belonging to 6 classes.


In [14]:
STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=valid_gen.n//valid_gen.batch_size
STEP_SIZE_TEST=test_gen.n//test_gen.batch_size



model.fit_generator(generator=train_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_gen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=30
)

C:\Users\nazar\AppData\Local\Temp\ipykernel_11472\4126350258.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_gen,


Epoch 1/30
251/251 [==============================] - 259s 1s/step - loss: 0.3463 - accuracy: 0.5810 - val_loss: 0.3145 - val_accuracy: 0.6329
Epoch 2/30
251/251 [==============================] - 286s 1s/step - loss: 0.1576 - accuracy: 0.8359 - val_loss: 0.2964 - val_accuracy: 0.7391
Epoch 3/30
251/251 [==============================] - 290s 1s/step - loss: 0.0958 - accuracy: 0.9197 - val_loss: 0.2747 - val_accuracy: 0.7643
Epoch 4/30
251/251 [==============================] - 294s 1s/step - loss: 0.0690 - accuracy: 0.9424 - val_loss: 0.3031 - val_accuracy: 0.7605
Epoch 5/30
251/251 [==============================] - 270s 1s/step - loss: 0.0549 - accuracy: 0.9508 - val_loss: 0.3350 - val_accuracy: 0.7545
Epoch 6/30
251/251 [==============================] - 275s 1s/step - loss: 0.0506 - accuracy: 0.9518 - val_loss: 0.3188 - val_accuracy: 0.7564
Epoch 7/30
251/251 [==============================] - 292s 1s/step - loss: 0.0433 - accuracy: 0.9550 - val_loss: 0.3090 - val_accuracy: 0.7590

In [15]:
acc=model.evaluate(test_gen)[1]* 100
print (acc)

23/23 [==============================] - 7s 296ms/step - loss: 0.1777 - accuracy: 0.9067
90.66852331161499


In [16]:
model.evaluate_generator(generator=valid_gen,
steps=STEP_SIZE_TEST)

C:\Users\nazar\AppData\Local\Temp\ipykernel_11472\2212924209.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(generator=valid_gen,


[0.5099822878837585, 0.75]

In [19]:
test_gen.reset()
pred=model.predict_generator(test_gen,
steps=STEP_SIZE_TEST,
verbose=1)

C:\Users\nazar\AppData\Local\Temp\ipykernel_11472\4150660490.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred=model.predict_generator(test_gen,


22/22 [==============================] - 7s 292ms/step


In [23]:
import numpy as np
predicted_class_indices=np.argmax(pred,axis=1)

In [24]:
model.save('military.h5')

In [25]:
loss, accuracy = model.evaluate(valid_gen)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

84/84 [==============================] - 38s 445ms/step - loss: 0.4903 - accuracy: 0.7621
Loss:  0.49030059576034546
Accuracy:  0.762135922908783


In [16]:
model = tf.keras.models.load_model('military.h5')

In [28]:
pred=model.predict_generator(test_gen,
steps=test_gen.n//test_gen.batch_size,
verbose=1)

C:\Users\nazar\AppData\Local\Temp\ipykernel_11472\2741743327.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred=model.predict_generator(test_gen,


22/22 [==============================] - 7s 288ms/step


In [26]:
import numpy as np

In [3]:
predicted_class_indices=np.argmax(pred,axis=1)
labels = (test_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]



NameError: name 'pred' is not defined

In [4]:
filenames=test_gen.filenames
results=pd.DataFrame.from_dict({"Filename":filenames,
                      "Predictions":predictions},orient='index')
results.transpose().to_csv("results.csv",index=False)

NameError: name 'test_gen' is not defined

In [5]:
print(len(test_gen.filenames))
print(len(predicted_class_indices))

NameError: name 'test_gen' is not defined

In [19]:
from tensorflow.keras.utils import load_img, img_to_array

In [20]:
def load_image(filename):
    try:
        print(filename)
        img = keras.utils.load_img(filename, target_size=(image_rows, image_cols))
        
        # convert to array
        img = img_to_array(img)
        # reshape into a single sample with 3 channels
        img = img.reshape(1, image_rows, image_cols, 3)
        # prepare pixel data
        img = img.astype('float32')
        img = img / 255.0
        return img
    except Exception as error:
        print(f"Exception = {error}")

In [21]:
#labels = (test_gen.class_indices)
labels = {   
                 'civilian aircraft': 0,
                 'civilian car': 1,
                 'military aircraft': 2,
                 'military helicopter': 3,
                 'military tank': 4,
                 'military truck': 5
              }

In [22]:
model = tf.keras.models.load_model('military.h5')

In [23]:
labels_ua = {   
             'civilian aircraft': 'цивільний літак',
             'civilian car': 'автомобіль',
             'military aircraft': 'військовий літак',
             'military helicopter': 'військовий вертоліт',
             'military tank': 'військовий танк',
             'military truck': 'військова вантажівка'
          }

In [30]:
def recognite(file):
    img = load_image(file)
    pred = model.predict(img,verbose = 0)
    #print(pred.round(3))
    predicted_class_indices=np.argmax(pred,axis=1)
    print(np.max(pred))
    print(predicted_class_indices[0])
    labels = {   
                 'civilian aircraft': 0,
                 'civilian car': 1,
                 'military aircraft': 2,
                 'military helicopter': 3,
                 'military tank': 4,
                 'military truck': 5
              }
    labels = dict((v,k) for k,v in labels.items())

    predictions = [labels_ua[labels[k]] for k in predicted_class_indices]

   

    #print(predictions[0])
    #print('\n')
    for i in predictions:
        print(i)
    #print(predictions[0])
    #return predictions[0]

In [31]:
recognite("./Images/000_1H95VB.width-800.jpg")
recognite("./машин.jpg")
recognite("./літак.jpg")
recognite("./гел.jpg")
recognite("./бтр.jpg")
recognite("./літак військ.jpg")


./Images/000_1H95VB.width-800.jpg
0.9999999
2
військовий літак
./машин.jpg
0.9940866
5
військова вантажівка
./літак.jpg
0.99996555
0
цивільний літак
./гел.jpg
1.0
3
військовий вертоліт
./бтр.jpg
0.99751246
4
військовий танк
./літак військ.jpg
0.9999999
2
військовий літак


In [ ]:
from os import walk
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg


f = []

for (dirpath, dirnames, filenames) in walk('./test_img'):    
    for i in filenames:
        #plt.imshow(mpimg.imread(dirpath+'/'+i))
        fig = plt.figure()
        ax = fig.add_subplot(1, 2, 1)
        imgplot = plt.imshow(mpimg.imread(dirpath+'/'+i))
        ax.set_title(recognite(dirpath+'/'+i))

In [53]:
for i in labels:
    print(i)

civilian aircraft
civilian car
military aircraft
military helicopter
military tank
military truck


In [4]:
import opencv

ModuleNotFoundError: No module named 'opencv'